# Import

In [1]:
import numpy as np
import random
import torch
from x_transformers import XTransformer
import torch
from torch.utils.tensorboard import SummaryWriter

In [2]:
# %load_ext tensorboard
# %tensorboard --logdir log

In [3]:
# !rm -r log_comp

In [4]:
writer = SummaryWriter(log_dir='log_comp')

def train_validate_model(model, batch_generator, optimizer, model_name, num_batches=1e4, eval_size=5):

    for i in range(num_batches):

        model.train()
        batch_generator.train()
        
        src, tgt, src_mask, tgt_mask = next(batch_generator)
        loss = model(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask)
        loss.backward()

        loss_value = loss.item()
        print(f'{i}: {loss_value}')
        
        writer.add_scalars("/train/loss", {model_name: loss_value}, i)
        if loss_value < 1e-3:
            break

        optim.step()
        optim.zero_grad()

        if i != 0 and i % GENERATE_EVERY == 0:
            model.eval()
            batch_generator.eval()

            src, tgt, src_mask, _ = next(batch_generator)
            src, tgt = src[:eval_size], tgt[:eval_size, 1:]
            start_tokens = tgt[:1, :1]
            sm = src_mask[:1]

            num_correct = 0
            total_batch_len = 0
            for s, t in zip(src, tgt):
                sample = model.generate(s[None], start_tokens, DEC_SEQ_LEN, src_mask=sm)
                num_correct += torch.abs(((t == sample) & (t != 0)).float()).sum()
                total_batch_len += (t != 0).float().sum()

            accuracy = num_correct / total_batch_len
            writer.add_scalars("/val/accuracy", {model_name: accuracy}, i)

            print(f"input:  ", s)
            print(f"predicted output:  ", sample)
            print(f"correct output:  ", t)
            print(f"accuracy: {accuracy}")

    writer.flush()

# 1. Copy

In [5]:
NUM_BATCHES = int(1e5)
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
GENERATE_EVERY = 100
NUM_TOKENS = 16 + 2
ENC_SEQ_LEN = 32//2
DEC_SEQ_LEN = 64//2

In [6]:
class copy_generator:
    def __init__(self):
        self.mode = 'train'
        self.X, self.y = np.load('data/copy_X.npy'), np.load('data/copy_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool().cuda()
        self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool().cuda()

    def train(self):
        self.mode = 'train'

    def eval(self):
        self.mode = 'eval'
    
    def __next__(self):
        if self.mode == 'train':
            X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
            y = np.zeros([BATCH_SIZE, DEC_SEQ_LEN+1]).astype(int)
            y[:, 0] = 1
            for i in range(BATCH_SIZE):
                sequence_length = np.random.randint(1, ENC_SEQ_LEN)
                random_sequence = np.random.randint(2, NUM_TOKENS, sequence_length)
                
                X[i, :sequence_length] = random_sequence
                y[i, 1: 2 * sequence_length + 1] = np.concatenate([random_sequence] * 2)
        else:
            X = self.X[self.data_ptr: self.data_ptr+BATCH_SIZE]
            y = self.y[self.data_ptr: self.data_ptr+BATCH_SIZE]
            self.data_ptr = (self.data_ptr + BATCH_SIZE) % self.data_size

        return torch.tensor(X).cuda(), torch.tensor(y).cuda(), self.src_mask, self.tgt_mask        

In [5]:
# src, tgt, src_mask, _ = next(cg)
# src, tgt = src[:1], tgt[:1]
# start_tokens = tgt[:1, :1]
# sm = src_mask[:1]

# copy_base_model.generate(src, start_tokens, DEC_SEQ_LEN, src_mask=sm).shape

In [7]:
model_parameters = {'dim': 512,
    'tie_token_embeds': True,
    'return_tgt_loss': True,
    'enc_num_tokens': NUM_TOKENS,
    'enc_depth': 3,
    'enc_heads': 8,
    'enc_max_seq_len': ENC_SEQ_LEN,
    'dec_num_tokens': NUM_TOKENS,
    'dec_depth': 3,
    'dec_heads': 8}

## Base transformer

In [7]:
copy_base_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN).cuda()

optim = torch.optim.Adam(copy_base_model.parameters(), lr=LEARNING_RATE)
cgen = copy_generator()

In [10]:
train_validate_model(copy_base_model, cgen, optim, 'copy_base', num_batches=2500)

26
713: 0.012518846429884434
714: 0.010278681293129921
715: 0.019673317670822144
716: 0.02574029751121998
717: 0.017987901344895363
718: 0.01122580748051405
719: 0.018426118418574333
720: 0.010502980090677738
721: 0.012259489856660366
722: 0.025608202442526817
723: 0.01579061709344387
724: 0.0057714954018592834
725: 0.008786213584244251
726: 0.006396178621798754
727: 0.010846256278455257
728: 0.02847670391201973
729: 0.010643781162798405
730: 0.029693400487303734
731: 0.011929447762668133
732: 0.01262326817959547
733: 0.023251276463270187
734: 0.019490109756588936
735: 0.018874013796448708
736: 0.008951031602919102
737: 0.013620666228234768
738: 0.030943507328629494
739: 0.018979601562023163
740: 0.006637816317379475
741: 0.006658220198005438
742: 0.012739619240164757
743: 0.005272559355944395
744: 0.016616294160485268
745: 0.02420584298670292
746: 0.02271551825106144
747: 0.017243247479200363
748: 0.010676784440875053
749: 0.023909809067845345
750: 0.014366304501891136
751: 0.03081054

## Memory transformer

In [8]:
copy_memory_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN, num_memory_tokens=16).cuda()

optim = torch.optim.Adam(copy_memory_model.parameters(), lr=LEARNING_RATE)
cgen = copy_generator()

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [56]:
train_validate_model(copy_memory_model, cgen, optim, 'copy_memory', num_batches=2500)

0: 2.8105568885803223
1: 5.373259544372559
2: 4.240912437438965
input:   tensor([ 4, 13,  8, 13, 12, 16,  3, 13, 16,  9, 15, 12,  0,  0,  0,  0])
predicted output:   tensor([[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]])
correct output:   tensor([ 1,  4, 13,  8, 13, 12, 16,  3, 13, 16,  9, 15, 12,  4, 13,  8, 13, 12,
        16,  3, 13, 16,  9, 15, 12,  0,  0,  0,  0,  0,  0,  0,  0])
accuracy: 0.014598540030419827
3: 3.4374091625213623
4: 3.011038064956665
input:   tensor([12,  5,  3,  5,  4, 16, 17, 13,  8,  5, 11,  5,  0,  0,  0,  0])
predicted output:   tensor([[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5]])
correct output:   tensor([ 1, 12,  5, 12,  5,  4, 16, 17, 13,  8,  5, 11,  5, 16, 14,  3,  5,  4,
        16, 17, 13,  8,  5, 11,  5,  0,  0,  0,  0,  0,  0,  0,  0])
accuracy: 0.0476190485060215
5: 2.807417154312134


KeyboardInterrupt: 

# 2. Reverse

In [58]:
NUM_BATCHES = int(1e5)
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
GENERATE_EVERY  = 3
NUM_TOKENS = 16 + 2
ENC_SEQ_LEN = 32
DEC_SEQ_LEN = 32

In [60]:
class reverse_generator:
    def __init__(self):
        self.mode = 'train'
        self.X, self.y = np.load('data/reverse_X.npy'), np.load('data/reverse_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool()#.cuda()
        self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool()#.cuda()

    def train(self):
        self.mode = 'train'

    def eval(self):
        self.mode = 'eval'
    
    def __next__(self):
        if self.mode == 'train':
            X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
            y = np.zeros([BATCH_SIZE, DEC_SEQ_LEN+1]).astype(int)
            y[:, 0] = 1
            for i in range(BATCH_SIZE):
                sequence_length = np.random.randint(1, ENC_SEQ_LEN)
                random_sequence = np.random.randint(2, NUM_TOKENS, sequence_length)
                
                X[i, :sequence_length] = random_sequence
                y[i, 1:sequence_length + 1] = random_sequence[::-1]
        else:
            X = self.X[self.data_ptr: self.data_ptr+BATCH_SIZE]
            y = self.y[self.data_ptr: self.data_ptr+BATCH_SIZE]
            self.data_ptr = (self.data_ptr + BATCH_SIZE) % self.data_size

        return torch.tensor(X), torch.tensor(y), self.src_mask, self.tgt_mask        

In [86]:
# rg = reverse_generator()
# X, y, sm, tm = next(rg)
# X, y, sm, tm

In [87]:
model_parameters = {'dim': 512,
    'tie_token_embeds': True,
    'return_tgt_loss': True,
    'enc_num_tokens': NUM_TOKENS,
    'enc_depth': 3,
    'enc_heads': 8,
    'enc_max_seq_len': ENC_SEQ_LEN,
    'dec_num_tokens': NUM_TOKENS,
    'dec_depth': 3,
    'dec_heads': 8}

## Base transformer

In [88]:
reverse_base_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN)#.cuda()

optim = torch.optim.Adam(reverse_base_model.parameters(), lr=LEARNING_RATE)
rgen = reverse_generator()

In [89]:
train_validate_model(reverse_base_model, rgen, optim, 'reverse_base', num_batches=601)

0: 3.000492572784424
1: 4.143182754516602
2: 6.0525641441345215
3: 3.476656675338745
input:   tensor([12, 17,  9, 11, 12,  4,  8,  8, 14,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
predicted output:   tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
correct output:   tensor([ 6, 14,  8,  8,  4, 12, 11,  9, 17, 12,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
accuracy: 0.0
4: 1.950818657875061
5: 1.8770551681518555
6: 2.4338696002960205
input:   tensor([13,  3, 14,  6,  4, 14, 14,  5, 11,  3, 17, 13,  4,  4, 17, 11,  9, 17,
         2,  8, 16,  8,  8,  6, 16, 16,  9,  0,  0,  0,  0,  0])
predicted output:   tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
correct output:   tensor([ 9, 16, 16,  6,  8,  8, 16,  8,  2, 17,  9, 11, 17,  4,  4, 13, 17,

KeyboardInterrupt: 

## Memory transformer

In [75]:
reverse_memory_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN, num_memory_tokens=16)#.cuda()

optim = torch.optim.Adam(reverse_memory_model.parameters(), lr=LEARNING_RATE)
rgen = reverse_generator()

In [76]:
train_validate_model(reverse_memory_model, rgen, optim, 'reverse_memory', num_batches=601)

0: 3.0095040798187256
1: 5.572384834289551
2: 3.79541277885437
3: 2.949702262878418
input:   tensor([ 8, 11, 11, 16,  9,  5, 11,  3, 13, 17, 13,  9, 10,  6,  9,  2,  7,  5,
        15, 17, 11,  6, 16, 17,  9, 15,  3,  7,  3,  5, 10,  0])
predicted output:   tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]])
correct output:   tensor([ 1, 10,  5,  3,  7,  3, 15,  9, 17, 16,  6, 11, 17, 15,  5,  7,  2,  9,
         6, 10,  9, 13, 17, 13,  3, 11,  5,  9, 16, 11, 11,  8,  0])
accuracy: 0.006802720949053764
4: 2.800572395324707
5: 2.860241174697876
6: 2.861840009689331
input:   tensor([ 5, 10,  5, 11, 16,  9,  7, 16, 15,  2, 10, 10, 10,  2, 16, 15,  6,  2,
        11, 13,  3,  2, 11,  6, 11, 11, 14,  8,  7,  5, 10,  0])
predicted output:   tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]])
correct output:   tensor([ 1, 11,  5, 10,  5,  2, 10, 10, 10,  2, 15,

KeyboardInterrupt: 

# Associative retrieval

In [65]:
def get_three_letters():
    return np.random.choice(range(0,26), 3, replace=False)

def get_three_numbers():
    return np.random.choice(range(26, 26+10), 3, replace=False)

def create_sequence(one_hot=True):
    letters = get_three_letters()
    numbers = get_three_numbers()
    X = np.zeros((9))
    y = np.zeros((1))
    for i in range(0, 5, 2):
        X[i] = letters[i//2]
        X[i+1] = numbers[i//2]

    # append ??
    X[6] = 26+10
    X[7] = 26+10

    # last key and respective value (y)
    index = np.random.choice(range(0,3), 1, replace=False)
    X[8] = letters[index]
    y = numbers[index]

    if one_hot:
        # one hot encode X and y
        X_one_hot = np.eye(26+10+1)[np.array(X).astype('int')]
        y_one_hot = np.eye(26+10+1)[y][0]

        return X_one_hot, y_one_hot
    else:
        return X, y

def ordinal_to_alpha(sequence):
    """
    Convert from ordinal to alpha-numeric representations.
    Just for funsies :)
    """
    corpus = ['a','b','c','d','e','f','g','h','i','j','k','l',
              'm','n','o','p','q','r','s','t','u','v','w','x','y','z',
               0, 1, 2, 3, 4, 5, 6, 7, 8, 9, '?']

    conversion = ""
    for item in sequence:
        conversion += str(corpus[item.argmax()])
    return conversion

In [66]:
NUM_BATCHES = int(1e5)
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
GENERATE_EVERY  = 3
NUM_TOKENS = 26+10+1
ENC_SEQ_LEN = 9
DEC_SEQ_LEN = 1

In [67]:
# def retrieval_generator():
#     X = np.zeros([BATCH_SIZE, 9]).astype(int)
#     y = np.zeros([BATCH_SIZE, 2]).astype(int)
#     y[:, 0] = 1

#     src_mask = torch.ones(BATCH_SIZE, 9).bool()#.cuda()
#     tgt_mask = torch.ones(BATCH_SIZE, 2).bool()#.cuda()

#     while(True):        
#         for i in range(BATCH_SIZE):
#             X[i], y[i, 1:] = create_sequence(one_hot=False)

#         yield torch.tensor(X), torch.tensor(y), src_mask, tgt_mask

In [68]:
class retrieval_generator:
    def __init__(self):
        self.mode = 'train'
        self.X, self.y = np.load('data/retrieval_X.npy'), np.load('data/retrieval_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool()#.cuda()
        self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool()#.cuda()

    def train(self):
        self.mode = 'train'

    def eval(self):
        self.mode = 'eval'
    
    def __next__(self):
        if self.mode == 'train':
            X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
            y = np.zeros([BATCH_SIZE, DEC_SEQ_LEN+1]).astype(int)
            y[:, 0] = 1
            for i in range(BATCH_SIZE):
                X[i], y[i, 1:] = create_sequence(one_hot=False)
        else:
            X = self.X[self.data_ptr: self.data_ptr+BATCH_SIZE]
            y = self.y[self.data_ptr: self.data_ptr+BATCH_SIZE]
            self.data_ptr = (self.data_ptr + BATCH_SIZE) % self.data_size

        return torch.tensor(X), torch.tensor(y), self.src_mask, self.tgt_mask        

In [69]:
model_parameters = {'dim': 512,
    'tie_token_embeds': True,
    'return_tgt_loss': True,
    'enc_num_tokens': NUM_TOKENS,
    'enc_depth': 3,
    'enc_heads': 8,
    'enc_max_seq_len': ENC_SEQ_LEN,
    'dec_num_tokens': NUM_TOKENS,
    'dec_depth': 3,
    'dec_heads': 8}

## Base transformer

In [89]:
# # retr_gen = retrieval_generator()
# # retr_gen.eval()
# src, tgt, sm, tm = next(retr_gen)
# src.shape, tgt.shape, sm.shape, tm.shape
# src, tgt, sm, tm

In [72]:
# retr_gen = retrieval_generator()
# src, tgt, sm, tm = next(retr_gen)
# src.shape, tgt.shape, sm.shape, tm.shape
# tgt

In [73]:
retrieval_base_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN)#.cuda()

optim = torch.optim.Adam(retrieval_base_model.parameters(), lr=LEARNING_RATE)
retr_gen = retrieval_generator()

In [74]:
train_validate_model(retrieval_base_model, retr_gen, optim, 'retrieval_base', num_batches=1000)

0: 3.6622865200042725
1: 2.652991771697998
2: 2.775786876678467
3: 3.0393452644348145
t:  tensor([28])
sample:  tensor([[27]])
correct mask =  tensor([[False]])
t:  tensor([33])
sample:  tensor([[31]])
correct mask =  tensor([[False]])
t:  tensor([32])
sample:  tensor([[30]])
correct mask =  tensor([[False]])
t:  tensor([33])
sample:  tensor([[30]])
correct mask =  tensor([[False]])
t:  tensor([32])
sample:  tensor([[30]])
correct mask =  tensor([[False]])
input:   tensor([14, 34, 20, 32, 16, 31, 36, 36, 20])
predicted output:   tensor([[30]])
correct output:   tensor([32])
accuracy: 0.0
4: 2.3778741359710693
5: 2.5743558406829834
6: 2.4867851734161377
t:  tensor([27])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([33])
sample:  tensor([[35]])
correct mask =  tensor([[False]])
t:  tensor([32])
sample:  tensor([[35]])
correct mask =  tensor([[False]])
t:  tensor([27])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([28])
sample:  tensor([[35]]

KeyboardInterrupt: 

## Memory transformer

In [75]:
retrieval_memory_model = XTransformer(**model_parameters, dec_max_seq_len=DEC_SEQ_LEN, num_memory_tokens=4)#.cuda()

optim = torch.optim.Adam(retrieval_memory_model.parameters(), lr=LEARNING_RATE)
retr_gen = retrieval_generator()

In [76]:
train_validate_model(retrieval_memory_model, retr_gen, optim, 'retrieval_memory', num_batches=1000, eval_size=20)

0: 3.7840535640716553
1: 3.407494306564331
2: 2.976376533508301
3: 3.039721727371216
t:  tensor([33])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([35])
sample:  tensor([[29]])
correct mask =  tensor([[False]])
t:  tensor([28])
sample:  tensor([[29]])
correct mask =  tensor([[False]])
t:  tensor([29])
sample:  tensor([[32]])
correct mask =  tensor([[False]])
t:  tensor([27])
sample:  tensor([[32]])
correct mask =  tensor([[False]])
t:  tensor([34])
sample:  tensor([[34]])
correct mask =  tensor([[True]])
t:  tensor([33])
sample:  tensor([[29]])
correct mask =  tensor([[False]])
t:  tensor([26])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([29])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([27])
sample:  tensor([[29]])
correct mask =  tensor([[False]])
t:  tensor([35])
sample:  tensor([[32]])
correct mask =  tensor([[False]])
t:  tensor([35])
sample:  tensor([[34]])
correct mask =  tensor([[False]])
t:  tensor([32])

KeyboardInterrupt: 

# ListOPS

In [91]:
4 + 4*3 + 3 * 3 * 4 + 3 * 3 * 3 * 4

160

In [90]:
MIN = "[MIN"
MAX = "[MAX"
MED = "[MED"
SUM_MOD = "[SM"
END = "]"

OPERATORS = [MIN, MAX, MED, SUM_MOD]
VALUES = range(10)

VALUE_P = 0.25
MAX_ARGS = 3
MAX_DEPTH = 10

In [91]:
def generate_tree(depth):
    if depth < MAX_DEPTH:
        r = random.random()
    else:
        r = 1

    if r > VALUE_P:
        value = random.choice(VALUES)
        return value
    else:
        num_values = random.randint(2, MAX_ARGS)
        values = []
        for _ in range(num_values):
            values.append(generate_tree(depth + 1))

        op = random.choice(OPERATORS)
        t = (op, values[0])
        for value in values[1:]:
            t = (t, value)
        t = (t, END)
    return t


def to_string(t, parens=False):
    if isinstance(t, str):
        return t
    elif isinstance(t, int):
        return str(t)
    else:
        if parens:
            return '( ' + to_string(t[0], parens) + ' ' + to_string(t[1], parens) + ' )'
        else:
            return to_string(t[0], parens) + ' ' + to_string(t[1], parens)# + ' '


op2token = dict(zip(list(np.arange(10).astype(str)) + OPERATORS + [END], range(2, 15+2)))

def to_tokens(t):
    string = to_string(t)
    tokens = list(map(lambda x: op2token[x], string.split(' ')))
    return tokens


def to_value(t):
    if not isinstance(t, tuple):
        return t
    l = to_value(t[0])
    r = to_value(t[1])
    if l in OPERATORS:  # Create an unsaturated function.
        return (l, [r])
    elif r == END:  # l must be an unsaturated function.
        if l[0] == MIN:
            return min(l[1])
        elif l[0] == MAX:
            return max(l[1])
        # elif l[0] == FIRST:
        #     return l[1][0]
        # elif l[0] == LAST:
        #     return l[1][-1]
        elif l[0] == MED:
            return int(np.median(l[1]))
        elif l[0] == SUM_MOD:
            return (np.sum(l[1]) % 10)
    elif isinstance(l, tuple):  # We've hit an unsaturated function and an argument.
        return (l[0], l[1] + [r])

In [92]:
NUM_BATCHES = int(1e5)
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
GENERATE_EVERY  = 3
NUM_TOKENS = 10+5+2
ENC_SEQ_LEN = 150
DEC_SEQ_LEN = 1

In [93]:
def listops_generator(max_depth=MAX_DEPTH):

    src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool()#.cuda()
    tgt_mask = torch.ones(BATCH_SIZE, 2).bool()#.cuda()
    
    while(True):
        X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
        y = np.ones([BATCH_SIZE, 2]).astype(int) * 2
        # y[:, 0] = 1

        for i in range(BATCH_SIZE):
            t = generate_tree(max_depth)
            tokens, value = to_tokens(t), to_value(t) 
            X[i, 0:len(tokens)], y[i, 1:] = tokens, value+2
            del t

        src, tgt = torch.tensor(X), torch.tensor(y)#.cuda()
        yield src, tgt, src_mask, tgt_mask

In [97]:
class listops_generator:
    def __init__(self, max_depth=MAX_DEPTH):
        self.max_depth = max_depth
        self.mode = 'train'
        self.X, self.y = np.load('data/listops_X.npy'), np.load('data/listops_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        self.src_mask = torch.ones(BATCH_SIZE, ENC_SEQ_LEN).bool()#.cuda()
        self.tgt_mask = torch.ones(BATCH_SIZE, DEC_SEQ_LEN+1).bool()#.cuda()

    def train(self):
        self.mode = 'train'

    def eval(self):
        self.mode = 'eval'
    
    def __next__(self):
        if self.mode == 'train':
            X = np.zeros([BATCH_SIZE, ENC_SEQ_LEN]).astype(int)
            y = np.ones([BATCH_SIZE, 2]).astype(int) * 2

            for i in range(BATCH_SIZE):
                t = generate_tree(self.max_depth)
                tokens, value = to_tokens(t), to_value(t) 
                X[i, 0:len(tokens)], y[i, 1:] = tokens, value+2
                del t

            src, tgt = torch.tensor(X), torch.tensor(y)#.cuda()
        else:
            X = self.X[self.data_ptr: self.data_ptr+BATCH_SIZE]
            y = self.y[self.data_ptr: self.data_ptr+BATCH_SIZE]
            self.data_ptr = (self.data_ptr + BATCH_SIZE) % self.data_size

        return torch.tensor(X), torch.tensor(y), self.src_mask, self.tgt_mask        

In [115]:
# lg = listops_generator(4)
# lg.eval()
# next(lg)

(tensor([[ 3,  0,  0,  ...,  0,  0,  0],
         [11,  0,  0,  ...,  0,  0,  0],
         [ 9,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 6,  0,  0,  ...,  0,  0,  0],
         [ 8,  0,  0,  ...,  0,  0,  0],
         [ 5,  0,  0,  ...,  0,  0,  0]]),
 tensor([[ 2,  3],
         [ 2, 11],
         [ 2,  9],
         [ 2,  9],
         [ 2, 11],
         [ 2,  8],
         [ 2,  8],
         [ 2,  5],
         [ 2,  9],
         [ 2,  7],
         [ 2,  8],
         [ 2,  7],
         [ 2,  6],
         [ 2,  6],
         [ 2,  7],
         [ 2,  7],
         [ 2,  9],
         [ 2,  6],
         [ 2,  5],
         [ 2,  8],
         [ 2,  8],
         [ 2,  4],
         [ 2, 10],
         [ 2,  8],
         [ 2,  3],
         [ 2,  9],
         [ 2,  9],
         [ 2, 10],
         [ 2,  9],
         [ 2,  6],
         [ 2,  8],
         [ 2,  5]]),
 tensor([[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, 

In [71]:
model_parameters = {'dim': 512,
    'tie_token_embeds': True,
    'return_tgt_loss': True,
    'enc_num_tokens': NUM_TOKENS,
    'enc_depth': 3,
    'enc_heads': 8,
    'enc_max_seq_len': ENC_SEQ_LEN,
    'dec_num_tokens': NUM_TOKENS,
    'dec_depth': 3,
    'dec_heads': 8}

## Base transformer

In [100]:
# del listops_base_model
# del tokens 
torch.cuda.empty_cache()

In [106]:
listops_base_model = XTransformer(**model_parameters, dec_max_seq_len=2)#.cuda()

optim = torch.optim.Adam(listops_base_model.parameters(), lr=LEARNING_RATE)
listops_gen = listops_generator(max_depth=4)


In [107]:
train_validate_model(listops_base_model, listops_gen, optim, 'listops_base', num_batches=150)

 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
predicted output:   tensor([[8]])
correct output:   tensor([8])
accuracy: 0.20000000298023224
70: 0.8611487150192261
71: 0.9114288091659546
72: 0.5899685621261597
input:   tensor([11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  

## Memory transformer

In [9]:
listops_memory_model = XTransformer(**model_parameters, dec_max_seq_len=2, num_memory_tokens=32).cuda()

optim = torch.optim.Adam(listops_memory_model.parameters(), lr=LEARNING_RATE)
listops_gen = listops_generator(max_depth=4)

,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0]], device='cuda:0')
predicted output:   tensor([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

In [ ]:
train_validate_model(listops_memory_model, listops_gen, optim, 'listops_memory', num_batches=1500)